In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sd20co001_image_dataset_for_skindiseases_dry_oily_normalskin_path = kagglehub.dataset_download('sd20co001/image-dataset-for-skindiseases-dry-oily-normalskin')

print('Data source import complete.')                                            

In [ ]:
def load_model(model_path, model_name, num_classes, device):
    """Loads a pre-trained model from a specified path."""
    model = get_model(model_name, num_classes, pretrained=False, device=device)
    # Load the entire checkpoint
    checkpoint = torch.load(model_path, map_location=device, weights_only=False)
    # Extract the model state dictionary
    model_state_dict = checkpoint.get('model_state', checkpoint) # Use .get with a default for flexibility
    model.load_state_dict(model_state_dict)
    model.eval()
    return model

def predict(model, dataloader, device):
    """Makes predictions on a dataset."""
    model.eval()
    all_preds = []
    with torch.no_grad():
        for images, _ in tqdm(dataloader, desc="Predicting"):
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).detach().cpu().numpy()
            all_preds.extend(preds.tolist())
    return all_preds

def evaluate(y_true, y_pred, all_labels):
    """Evaluates predictions and displays classification report and confusion matrix."""
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=all_labels))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(15, 10))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=all_labels, yticklabels=all_labels)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()

# Load Model, Predict, and Evaluate

In [ ]:
# # Assuming you have saved your best model to "best_skin_model.pth"
# # Replace "best_skin_model.pth" with the actual path to your saved model
# model_path = "best_skin_model.pth" # REPLACE THIS WITH YOUR MODEL PATH
# best_model_name = "efficientnet_b2" # REPLACE THIS WITH THE MODEL NAME FROM YOUR BEST TRIAL
# num_classes = len(cls_to_idx)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# all_labels = sorted(cls_to_idx.keys()) # Get labels in the correct order

# try:
#     model = load_model(model_path, best_model_name, num_classes, device)
#     print(f"Model loaded successfully from {model_path}")

#     # Get true labels from the test dataset
#     y_true_test = [label for _, label in test_dataset]

#     # Make predictions on the test set
#     y_pred_test = predict(model, test_dataloader, device)

#     # Evaluate the model
#     evaluate(y_true_test, y_pred_test, all_labels)

# except FileNotFoundError:
#     print(f"Error: Model file not found at {model_path}. Please make sure the path is correct.")
# except Exception as e:
#     print(f"An error occurred: {e}")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sd20co001/image-dataset-for-skindiseases-dry-oily-normalskin")

print("Path to dataset files:", path)

# REQ LIBS NAD PACKAGES

In [ ]:
!pip install optuna -q

In [ ]:
!pip install wandb -q

In [ ]:
import torch
import os
import cv2
from pathlib import Path
from torchvision import transforms
from torchvision.transforms import v2
from PIL import Image
import pandas as pd
from torch.utils.data import DataLoader , Dataset , random_split , WeightedRandomSampler
from sklearn.model_selection import train_test_split
import glob
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , confusion_matrix
import torch.optim as optim
from tqdm import tqdm # progress bars
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from torch import nn
from torchvision import models
import timm
import math
from ast import Pass
import time
import copy
from collections import Counter
from optuna.pruners import MedianPruner # early stopping for bad trials.
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingLR, ReduceLROnPlateau
from sklearn.metrics import f1_score
import optuna # hyperparameter tuning engine
import wandb
import csv

*Device Agnostic Code*

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

*Secrets To be here*

In [ ]:
class user_secret:
    pass
    # user_secrets = UserSecretsClient()
    # secret_value_0 = user_secrets.get_secret("wandb_api_key")
    # wandb.login(key=wandb_api_key)
    # wandb.login()

In [ ]:
!nvidia-smi

In [ ]:
ROOT_DIR = "/root/.cache/kagglehub/datasets/sd20co001/image-dataset-for-skindiseases-dry-oily-normalskin/versions/1"



image_paths = glob.glob(ROOT_DIR + "/**/*.jpg",recursive=True)
image_paths += glob.glob(ROOT_DIR + "/**/*.jpeg",recursive=True)
image_paths += glob.glob(ROOT_DIR + "/**/*.png",recursive=True)
df = pd.DataFrame({
    "image_path" : image_paths,
    "label" : [os.path.basename(os.path.dirname(p)) for p in image_paths]
})
print("Total images:", len(df))
df.sample(10)

In [ ]:
df["label"].value_counts()

## Label_Mapping for similer disease**

In [ ]:
Label_Map = {
    "Cellulitis_Impetigo_And_Other_Bacterial_Infections": "Bacterial_Infections_Cellulitis_Impetigo",
    "Ba_Cellulitis": "Bacterial_Infections_Cellulitis_Impetigo",
    "Ba_Impetigo": "Bacterial_Infections_Cellulitis_Impetigo",

    "Ringworm": "Fungal_Infections_Ringworm_Candidiasis",
    "Athlete_Foot": "Fungal_Infections_Ringworm_Candidiasis",
    "Tinea_Ringworm_Candidiasis_And_Other_Fungal_Infections": "Fungal_Infections_Ringworm_Candidiasis",

    "Seborrheic_Keratoses_And_Other_Benign_Tumors": "Benign_Tumors_Keratoses",
    "Benign_Tumors": "Benign_Tumors_Keratoses",
    "Benign_Keratosis": "Benign_Tumors_Keratoses",

    "Malignant_Tumors": "Skin_Cancer_Malignant",
    "Malignant_Lesions": "Skin_Cancer_Malignant",
    "Basal_Cell_Carcinoma_and_Other_Carcinoma": "Skin_Cancer_Malignant",
    "Melanoma_Skin_Cancer_Nevi_And_Moles": "Skin_Cancer_Malignant",

    "Actinic_Keratosis": "Actinic_Keratosis_And_Cheilitis",
    "Actinic_Cheilitis": "Actinic_Keratosis_And_Cheilitis",
}
print(f"Lebel_counts_before_marging : {df.label.nunique()}")
df["label"] = df.label.replace(Label_Map)
print(f"Lebel_counts_AFTER_marging : {df.label.nunique()}")

In [ ]:
 # Saving The new CSV file
CSV_PATH = "skin_dataset.csv"
df.to_csv(CSV_PATH, index=False)
print("New_df_saved>>....")

# Split into train / validation / test

In [ ]:
X = df["image_path"]
y = df["label"]

X_train, X_temp , y_train , y_temp = train_test_split(
    X,
    y,
    test_size = 0.25,
    stratify = y ,
    random_state = 23
)

X_val , X_test , y_val , y_test = train_test_split(
    X_temp ,
    y_temp,
    test_size = 0.5,
    stratify = y_temp,
    random_state = 23
)
print(f"Train ; {X_train.shape} , Test : {X_test.shape} , Val : {X_val.shape}")

In [ ]:
df["label"].unique()

# Augmentation and Transformation

In [ ]:
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

train_transform = v2.Compose([
    v2.Resize((380,380)),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ColorJitter(brightness = 0.2,
                  contrast = 0.2,
                  saturation = 0.3),
    v2.RandomVerticalFlip(p = 0.5),
    v2.RandomRotation(25),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(imagenet_mean,
                imagenet_std)
])

val_transform = v2.Compose([
    v2.Resize((380,380)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(imagenet_mean,imagenet_std)
])

test_transform = v2.Compose([
    v2.Resize((380,380)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale= True),
    v2.Normalize(imagenet_mean,
                imagenet_std)
])
print(f"train {train_transform} , val : {val_transform} , test : {test_transform}")

# Dataset Creation

In [ ]:
train_df = pd.DataFrame({
    "image_path" : X_train,
    "label" : y_train
})
val_df = pd.DataFrame({
    "image_path" : X_val,
    "label" : y_val
})
test_df = pd.DataFrame({
    "image_path" : X_test,
    "label" : y_test
})

SAVE_DIR = "/kaggle/working/"

os.makedirs(SAVE_DIR, exist_ok=True)

train_df.to_csv(os.path.join(SAVE_DIR , "train.csv") , index = False)
val_df.to_csv(os.path.join(SAVE_DIR, "val.csv") , index = False)
test_df.to_csv(os.path.join(SAVE_DIR , "test.csv") , index = False)

print(f"Saved train.csv, val.csv, test.csv in {SAVE_DIR}")
print(f"Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}")

# Class Weight Computation (Coz Imbalance dataset)

In [ ]:
all_labels = sorted(train_df["label"].unique())
cls_to_idx = {label : idx for idx , label in enumerate(all_labels)}

class_counts = train_df["label"].value_counts().to_dict()
counts = np.array([class_counts.get(label, 0) for label in all_labels])

# Class-Balanced Loss Based on Effective Number of Samples ---> (CVPR 2019).
beta = 0.9999
effective_num = 1.0 - np.power(beta, counts)
weights = (1.0 - beta) / effective_num
weights = weights / np.sum(weights)

class_weights = torch.tensor(weights, dtype=torch.float32)

sample_weights = [class_weights[cls_to_idx[label]] for label in train_df["label"]]

# diyctionary for reference
normalized_weights = {
    cls: w for cls, w in zip(class_counts.keys(), weights)
}

print("✅ Class-balanced Weights:")
for k, v in normalized_weights.items():
    print(f"{k}: {v:.6f}")

# DataLoader -------loadingggggggggggggggggg..........

In [ ]:
class SKINDISEASE(Dataset):
    def __init__(self,df, transforms = None , label_dict = None):
        self.data = df
        self.transforms = transforms
        self.label_dict = label_dict
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        image_path = self.data.iloc[idx]["image_path"]
        label = self.data.iloc[idx]["label"]

        if self.label_dict:
            label = self.label_dict[label]
        img = Image.open(image_path).convert("RGB")

        if self.transforms :
            img = self.transforms(img)
        return img , label # WHY ?? int(label) -> Look at the next cell

## Dataset

In [ ]:
train_dataset = SKINDISEASE(
    df = train_df,
    transforms = train_transform,
    label_dict = cls_to_idx
)
val_dataset = SKINDISEASE(
    df = val_df ,
    transforms = val_transform ,
    label_dict = cls_to_idx
)
test_dataset = SKINDISEASE(
    df = test_df,
    transforms = test_transform ,
    label_dict = cls_to_idx
)
display(train_dataset , val_dataset , test_dataset)

## Dataloader

In [ ]:
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_dataloader = DataLoader(
                          dataset = train_dataset,
                          batch_size=32,
                          sampler=sampler,
                          num_workers = 2,
                          pin_memory = True)
val_dataloader = DataLoader(
                          dataset = val_dataset ,
                          batch_size = 32,
                          shuffle = False,
                          num_workers = 2 ,
                          pin_memory = True
)
test_dataloader = DataLoader(
                          dataset = test_dataset ,
                          batch_size = 32,
                          shuffle = False,
                          num_workers = 2 ,
                          pin_memory = True
)
print(f"Train_dataloader length : {len(train_dataloader)}  | test_data_loader : {len(val_dataloader)} Test DataLoader length : {len(test_dataloader)}")


In [ ]:
len(cls_to_idx), train_df["label"].nunique()

In [ ]:
num_classes = len(cls_to_idx)
num_classes

# Model Factory

In [ ]:
def get_model(name ,
             num_classes ,
             pretrained = True,
             device = "cuda"):
    if name.startswith("efficientnet_b"):
        if pretrained:
            # Correctly specified the weight enum
            if name == "efficientnet_b2":
                weights = models.EfficientNet_B2_Weights.IMAGENET1K_V1
            # Added similar conditions for other efficientnet_b models if needed
            else:
                 weights = None # Or handle other efficientnet versions

            model = getattr(models , name)(weights=weights)
        else:
            model = getattr(models , name)(weights=None) # Passed weights=None for non-pretrained
        if hasattr(model , "classifier") :
            if isinstance (model.classifier , nn.Sequential) and isinstance(model.classifier[-1] , nn.Linear):
                in_f = model.classifier[-1].in_features
                model.classifier[-1] = nn.Linear(in_f , num_classes)

            else :
                in_f = model.classifier.in_features
                model.classifier = nn.Linear(in_f , num_classes)

    elif name == "densenet121" :
        model = models.densenet121(weights = models.DenseNet121_Weights.IMAGENET1K_V1 if pretrained else None)
        in_f = model.classifier.in_features
        model.classifier = nn.Linear(in_f , num_classes)
    elif name == "resnet50" :
        model = models.resnet50(weights = models.ResNet50_Weights.IMAGENET1K_V1 if pretrained else None)
        in_f = model.fc.in_features
        model.fc = nn.Linear(in_f , num_classes)
    else:
        raise ValueError(f"Unknown model : {name}")

    return model.to(device)

# Training and Validation Loop

In [ ]:
from tqdm.auto import tqdm # Use tqdm.auto for better environment compatibility

def train_one_epoch(model,
                   dataloader,
                   loss_fn,
                   optimizer,
                   device,
                   scheduler=None):
    model.train()
    running_loss = 0.0
    all_preds , all_labels = [] , []

    loop = tqdm(dataloader,desc="Train", leave=False) # Use leave=False to remove the bar after completion
    for images , labels in loop :
        images , labels = images.to(device) , labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()

        if scheduler and isinstance(scheduler,OneCycleLR):
            scheduler.step()

        running_loss += loss.item() * images.size(0)
        preds = torch.argmax(outputs , dim = 1).detach().cpu().numpy()
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.detach().cpu().numpy().tolist())

        loop.set_postfix(loss=loss.item())


    avg_loss = running_loss / len(dataloader.dataset)
    return avg_loss , all_labels , all_preds

def validate(model ,
            dataloader ,
            loss_fn ,
            device):
    model.eval()
    running_loss = 0.0
    all_labels , all_preds = [], []

    with torch.no_grad():
        loop = tqdm(dataloader,desc="Val", leave=False) # Use leave=False
        for images , labels in loop :
            images , labels = images.to(device) , labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs , labels)


            running_loss += loss.item() * images.size(0)
            preds = torch.argmax(outputs, dim = 1).detach().cpu().numpy()
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.detach().cpu().numpy().tolist())
            loop.set_postfix(loss=loss.item())


        avg_loss = running_loss / len(dataloader.dataset)
        return avg_loss , all_labels , all_preds

In [ ]:
try:
    HAS_WANDB = True
except Exception:
    HAS_WANDB = False
HAS_WANDB

# Helper Functions

In [ ]:
def get_labels_from_dataset(dataset):
    """
    Try to detect labels list for common dataset types (ImageFolder, custom).
    """

    if hasattr(dataset , "targets"):
        return list(dataset.targets)
    if hasattr(dataset,"labels"):
        return list(dataset.labels)
    if hasattr(dataset,"samples"):
        # samples is a list of (image_path, label)
        return [label for _, label in dataset.samples]

    labels = []
    # the dataset returns (image, label) pairs
    for _, lbl in dataset:
        labels.append(int(lbl))
    return labels

def get_weighted_sampler(dataset):
    labels = get_labels_from_dataset(dataset)
    class_counts = Counter(labels)
    num_samples = len(labels)
    weights = [1.0 / class_counts[lbl] for lbl in labels]
    sampler = WeightedRandomSampler(weights,
                                   num_samples = num_samples,
                                   replacement = True)
    return sampler

def get_macro_f1(y_true, y_pred , num_classes):
    return f1_score(y_true,y_pred,average="macro",labels=list(range(num_classes)))

# Optuna Objective Function

In [ ]:
def objective(trial,
              train_dataset,
              val_dataset,
              device,
              class_weights,
              num_classes,
              batch_size=32,
              seeds=(24, 2021),
              epochs=3):

    # --- Hyperparameters to tune ---
    model_name = trial.suggest_categorical("model_name",
                                           ["efficientnet_b2", "densenet121", "resnet50"])
    optimizer_name = trial.suggest_categorical("optimizer",
                                               ["AdamW", "SGD"])
    lr = trial.suggest_float("lr",
                             1e-5, 1e-3,
                             log=True)
    weight_decay = trial.suggest_float("weight_decay",
                                       1e-6, 1e-3,
                                       log=True)

    if optimizer_name == "SGD":
        momentum = trial.suggest_float("momentum", 0.85, 0.99)

    scheduler_name = trial.suggest_categorical("scheduler",
                                               ["OneCycleLR", "CosineAnnealingLR", "ReduceLROnPlateau", "None"])
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 24, 32]) # Added batch size 8

    # --- Dataloaders ---
    train_sampler = get_weighted_sampler(train_dataset)
    val_dataloader = DataLoader(val_dataset,
                                batch_size=batch_size,
                                shuffle=False,
                                num_workers=2, # Reduced num_workers
                                pin_memory=True)

    seed_metrics = []
    for seed in seeds:
        set_seed(seed)

        train_dataloader = DataLoader(train_dataset,
                                      batch_size=batch_size,
                                      sampler=train_sampler,
                                      num_workers=2, # Reduced num_workers
                                      pin_memory=True)

        model = get_model(model_name,
                          num_classes=num_classes,
                          pretrained=True,
                          device=device)

        loss_fn = nn.CrossEntropyLoss(weight=class_weights.to(device))

        if optimizer_name == "AdamW":
            optimizer = optim.AdamW(model.parameters(),
                                    lr=lr,
                                    weight_decay=weight_decay)
        else:
            optimizer = optim.SGD(model.parameters(),
                                  lr=lr,
                                  momentum=momentum,
                                  weight_decay=weight_decay)

        # --- Scheduler ---
        scheduler = None
        if scheduler_name == "OneCycleLR":
            scheduler = OneCycleLR(optimizer,
                                   max_lr=lr,
                                   steps_per_epoch=len(train_dataloader),
                                   epochs=epochs)
        elif scheduler_name == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
        elif isinstance(scheduler, ReduceLROnPlateau): # Corrected this line
            scheduler = ReduceLROnPlateau(optimizer, mode="max", patience=2, factor=0.5)


        best_seed_f1 = 0.0
        for epoch in range(epochs):
            train_loss, _, _ = train_one_epoch(model=model,
                                               dataloader=train_dataloader,
                                               loss_fn=loss_fn,
                                               optimizer=optimizer,
                                               device=device,
                                               scheduler=scheduler)

            if scheduler and not isinstance(scheduler, (OneCycleLR, ReduceLROnPlateau)):
                scheduler.step()

            val_loss, y_true, y_pred = validate(model=model,
                                                dataloader=val_dataloader,
                                                loss_fn=loss_fn,
                                                device=device)
            macro_f1 = get_macro_f1(y_true, y_pred, num_classes)
            # print(f"[Trial {trial.number} | Epoch {epoch+1}/{epochs}] "f"Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | F1={macro_f1:.4f}")


            if isinstance(scheduler, ReduceLROnPlateau):
                scheduler.step(macro_f1)

            # Optuna reporting
            trial.report(macro_f1, epoch)
            if trial.should_prune():
                del model, optimizer, scheduler
                torch.cuda.empty_cache()
                raise optuna.TrialPruned()

            best_seed_f1 = max(best_seed_f1, macro_f1)

        seed_metrics.append(best_seed_f1)
        del model, optimizer, scheduler
        torch.cuda.empty_cache()

        print(f"✅ Trial {trial.number} finished with F1={best_seed_f1:.4f} | "
              f"lr={lr:.2e}, wd={weight_decay:.2e}, batch={batch_size}")

    return float(sum(seed_metrics) / len(seed_metrics))

In [ ]:
import os

def run_optuna_study(train_dataset, val_dataset, num_classes, class_weights, device,
                     n_trials=30, epochs=8, study_name="skin_tune", storage=None, use_wandb=False):
    # Remove the existing study database if it exists
    if storage and storage.startswith("sqlite:///"):
        db_path = storage.replace("sqlite:///", "")
        if os.path.exists(db_path):
            print(f"Removing existing Optuna study database: {db_path}")
            os.remove(db_path)


    pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=1, interval_steps=1)
    study = optuna.create_study(direction="maximize", study_name=study_name,
                                pruner=pruner, storage=storage, load_if_exists=True)

    if use_wandb and HAS_WANDB:
        wandb.init(project="optuna-hpo", reinit=True)

    func = lambda trial: objective(trial, train_dataset, val_dataset, device, class_weights, num_classes, epochs=epochs)
    study.optimize(func, n_trials=n_trials, gc_after_trial=True, show_progress_bar=True)

    print("Best trial:", study.best_trial.params, "score:", study.best_trial.value)

    if use_wandb and HAS_WANDB:
        wandb.finish(wait=True)

    return study

# Subset for hyperparams

In [ ]:
def get_balanced_subset_df(df, n_per_class=50, seed=42):
    """
    Returns a balanced subset of a DataFrame with exactly n_per_class samples per class.
    """
    balanced_df = df.groupby('label', group_keys=False).apply(
        lambda x: x.sample(n=min(len(x), n_per_class), random_state=seed)
    )
    return balanced_df.reset_index(drop=True)


# final Train

In [ ]:
def set_seed(seed):
    """Set seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
if __name__ == "__main__":
    train_dataset_full = train_dataloader.dataset
    val_dataset_full = val_dataloader.dataset
    num_classes = len(cls_to_idx)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
    print("11")
    train_df_reduced = get_balanced_subset_df(train_df, n_per_class=120, seed=42)
    print("train_loaded")
    val_df_reduced   = get_balanced_subset_df(val_df, n_per_class=30, seed=123)
    print("val_loaded")
    train_dataset_reduced = SKINDISEASE(
        df = train_df_reduced,
        transforms = train_transform,
        label_dict = cls_to_idx
    )
    val_dataset_reduced = SKINDISEASE(
        df = val_df_reduced ,
        transforms = val_transform ,
        label_dict = cls_to_idx
    )


    study = run_optuna_study(train_dataset_reduced, val_dataset_reduced, num_classes, class_weights_tensor,
                             device, n_trials=20, epochs=3,
                             study_name="skin_hpo_production",
                             storage="sqlite:///optuna_skin.db", use_wandb=True)

    # best_params = study.best_trial.params
    # saved_path = final_retrain_and_save(best_params, train_dataset, val_dataset,
    #                                     num_classes, class_weights, device,
    #                                     epochs=30,
    #                                     batch_size=best_params.get("batch_size", 24),
    #                                     save_path="best_skin_model.pth")
    # print("Final saved model:", saved_path)